#### Realiza tratamento dos dados para empresas_estabelecimentos

In [1]:
# Importa Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, LongType, Row
import json
from graphframes import *

In [2]:
spark = SparkSession.builder.appName("trata_dados").getOrCreate()

In [11]:
# Definindo o esquema para leitura do CSV
schema = StructType([
    StructField("nu_cnpj_raiz", StringType(), True),
    StructField("te_dados_em", StringType(), True),
    StructField("id_estabelecimento", StringType(), True),
    StructField("nu_cnpj_raiz_es", StringType(), True),
    StructField("te_dados_es", StringType(), True)
])

# Lendo o CSV com o esquema definido
df = spark.read \
    .option("delimiter", ",") \
    .option("multiline", "true") \
    .option("escape", "\"") \
    .csv("dataset/empresas_estabelecimentos.csv", schema=schema, header=True)

In [12]:
def extract_attributes(row):
    data_em = json.loads(row["te_dados_em"])
    data_es = json.loads(row["te_dados_es"])
    
    # Adicionando os atributos nu_cnpj_raiz e id_estabelecimento
    data_em['nu_cnpj_raiz'] = row['nu_cnpj_raiz']
    data_em['id_estabelecimento'] = row['id_estabelecimento']
    
    # Verificando se 'cnaesSecundarias' está presente em data_es
    if 'cnaesSecundarias' not in data_es:
        data_es['cnaesSecundarias'] = ''  # Adicionando o valor padrão vazio
    
    return Row(**data_em, **data_es)

df_emp_es = df.rdd.map(extract_attributes).toDF()

In [ ]:
# print (df_emp_es.head(2))
df_emp_es.show(truncate=False)
# df_emp_es.printSchema()

#### Realiza leitura de socios

In [8]:
# Definindo o esquema para leitura do CSV
schema = StructType([
    StructField("nu_cnpj_raiz", LongType(), True),
    StructField("id_socio", LongType(), True),
    StructField("te_dados", StringType(), True)
])

# Lendo o CSV com o esquema definido
df = spark.read \
    .option("delimiter", ",") \
    .option("multiline", "true") \
    .option("escape", "\"") \
    .csv("dataset/socios.csv", schema=schema, header=True)

In [9]:
def extract_attributes(row):
    data_em = json.loads(row["te_dados"])
    
    # Adicionando os atributos nu_cnpj_raiz e id_estabelecimento
    data_em['nu_cnpj_raiz'] = row['nu_cnpj_raiz']
    data_em['id_socio'] = row['id_socio']
    
    return Row(**data_em)

df_socios = df.rdd.map(extract_attributes).toDF()

In [10]:
df_socios.show(truncate=False)

+----+----------------+----------------+-----------------+------------------+---------------------+------------------------------+------------+------------+
|pais|entradaSociedade|socioEstrangeiro|qualificacaoSocio|identificadorSocio|cpfRepresentanteLegal|qualificacaoRepresentanteLegal|nu_cnpj_raiz|id_socio    |
+----+----------------+----------------+-----------------+------------------+---------------------+------------------------------+------------+------------+
|000 |20070110        |                |49               |2                 |00000000000          |00                            |3230097     |563224703   |
|000 |19990615        |                |49               |2                 |00000000000          |00                            |3230097     |26967782720 |
|000 |19990615        |                |49               |2                 |00000000000          |00                            |3230099     |6039729856  |
|000 |19990615        |                |22               |

In [14]:
spark.stop()